In [1]:
import sys

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

print(sys.executable)

/home/mehrzad/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/mehrzad/anaconda3/envs/dbpedia/bin/python


#
# Falcon 1B

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"


checkpoint = "tiiuae/falcon-rw-1b"


tokenizer = AutoTokenizer.from_pretrained(checkpoint)
pipeline = transformers.pipeline(
    "text-generation",
    model=checkpoint,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map=device,
)

Loading tiiuae/falcon-rw-1b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-rw-1b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y
Loading tiiuae/falcon-rw-1b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-rw-1b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y
Loading tiiuae/falcon-rw-1b requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-rw-1b. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'FalconForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'Pegasu

In [3]:
%%time

prompt = "SPARQL Query for: what is the capital of Japan?"

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: SPARQL Query for: what is the capital of Japan?
- I need the capital of Japan in English (with the Japanese characters, please).
- I'm a beginner in English and I need to find out the capital of Japan
- I have to find out the capital of Japan
- I have to find the capital of Japan
- I want to find the capital of Japan
- I would like to see the capital of Japan
- I'm looking for the capital of Japan
- I need to find out the capital of Japan
- I need to find out the capital of Japan
- I need to find out the capital of Japan
- I need to find out the capital of Japan
- I need to find out the capital of Japan
- I need to find out the capital of Japan
- I'm trying to find out the capital of Japan
- I'm trying to find out the capital of Japan
- I'm trying to find out the capital of Japan
- I'm trying to find out the capital of Japan
- I'm trying to find out the capital of Japan
- I'm trying to find out the capital of Japan
- I'm trying to find out the capital of Japan
- I'm trying to f

In [4]:
%%time

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}


input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}


input= SPARQL Query What is the capital of China?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}
"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}


input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}


input= SPARQL Query What is the capital of China?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}
}
}
This output is from a SQL query on DBpedia’s ontology of Germany. The first query shows the capital as Berlin. The second query shows the capital as Frankfurt. These two examples show that the German capital and the Chinese capital are not the same. However, they are both correct.
The German capital, Berlin, is the correct German capital; therefore, i

In [5]:
%%time

prompt = """
input= SPARQL Query What is the capital of France?

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query What is the capital of France?


input= SPARQL Query What is the value of the year?
What is a good example of an RDF graph?
The best way to learn how to use RDF is to start with an RDF Graph. In this example, we will create a simple SPARQL query that will return the value of the year. This graph is an RDF graph.
How do you use graphviz to create a RDF graph?
Here is a simple Graph.
How do you create an RDF graph in R?
To create a RDF graph, you need two files: a source file for the data (a list of values or objects), and a target file for the RDF graph. This target file must be a text document. The following example shows how to do that.
What are RDF graphs?
In a nutshell, RDF graphs are a representation of data. This can be a data set, a data structure (e.g. a relational table), or an RDF triplestructure.
How does RDF work?
RDF (Resource Description Framework) is an OWL-based standard that aims to provide a common vocabulary to describe web pages, document

In [6]:
%%time

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}


input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Iran dbo:capital ?capital .
}


input= SPARQL Query What is the capital of China?
out

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}


input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Iran dbo:capital ?capital .
}


input= SPARQL Query What is the capital of China?
out


input= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT?capital 
WHERE {
?capital {
 }
}
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT?capital 
WHERE {
!?capital }
}
input= PREFIX dbr: <http://dbpedia.org/resource/>
SELECT?capital 
WHERE {
?capital {
 }
}
Output from the query above:
<?p="{?capital}"?>?capital
The query outputs: <?p="{?capital}"?>?capital
The first element of the?p array is the value for the?capital field of the resource.
The first element o

In [7]:
%%time

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}


input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Iran dbo:capital ?capital .
}


input= SPARQL Query What is the capital of China?

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}


input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Iran dbo:capital ?capital .
}


input= SPARQL Query What is the capital of China?


input= SPARQL Query For what is the capital of China?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT?capital 
WHERE {
  dbr:China dbo:capital?capital.
}
}
The capital of Germany is Berlin, and the capital of Iran is Teheran.
- Posted using Chat Catcher
- Login to post comments
-
CPU times: user 1.38 s, sys: 0 ns, total: 1.38 s
Wall time: 1.38 s


In [8]:
%%time

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

 dbr:Japan dbo:capital?capital.
}
}
CPU times: user 204 ms, sys: 0 ns, total: 204 ms
Wall time: 203 ms


In [9]:
%%time

prompt = """
input= SPARQL Query for what is the capital of Belgium?
output=

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Belgium?
output=


output-
output-
output-
Output 1: the capital of Belgium
output-
output-
output=
output=
output=
Output 2: the capital of Belgium in a different format
output-
output=
output-
output=
output=
Output 3: a different format for the capital of Belgium
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
Output 4: what is the capital of France?
output=
output=
output=
input=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
input= SPARQL Query for what is the capital of France?
output=
output=
Output 5: the capital of France in a different format
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=
output=


In [10]:

prompt = """
input= SPARQL Query for what is the population of Japan?
output=
"""

sequences = pipeline(
    prompt,
    max_length=100,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the population of Japan?
output=
> SELECT *
> FROM population WHERE (
> (
> input= “Japan” or input= “Japan” )
> or
> (
> input= “Japan” or input= “Japan” )
> and
> (
> input= "Japan" or input= "Japan") or
> input= "Japan"


In [11]:

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for what is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query for what is the population of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?population 
WHERE {
  dbr:Japan dbo:populationTotal ?population .
}

input= SPARQL Query for what is the population of France?
output=

"""

sequences = pipeline(
    prompt,
    max_length=500,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id = 50256
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for what is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query for what is the population of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?population 
WHERE {
  dbr:Japan dbo:populationTotal ?population .
}

input= SPARQL Query for what is the population of France?
output=




In [12]:
tokenizer.eos_token_id


50256

In [13]:

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for what is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query for what is the population of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?population 
WHERE {
  dbr:Japan dbo:populationTotal ?population .
}

input= SPARQL Query for what is the population of France?
output=

"""

sequences = pipeline(
    prompt,
    max_length=500,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id = 50256
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for what is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query for what is the population of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?population 
WHERE {
  dbr:Japan dbo:populationTotal ?population .
}

input= SPARQL Query for what is the population of France?
output=



input= SPARQL Query for what is the population of Germany?
output= ?population 
WHERE {
?population 
}

input= SPARQL Query for what is the capital of Austria?
output= 
}

output= SELECT?capital 
WHERE {
?capital {
?capital {
?capital {
?population 
}
}
}
}
input= SPA

In [14]:
len(tokenizer(prompt)['input_ids'])

245

In [15]:

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for what is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query for what is the population of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?population 
WHERE {
  dbr:Japan dbo:populationTotal ?population .
}

input= SPARQL Query for what is the population of France?
output=

"""

sequences = pipeline(
    prompt,
    max_length=300,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id = 50256
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

/home/mehrzad/anaconda3/envs/dbpedia/lib/python3.10/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for what is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query for what is the population of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?population 
WHERE {
  dbr:Japan dbo:populationTotal ?population .
}

input= SPARQL Query for what is the population of France?
output=

 dba:France dbr:Population 
WHERE {
  dbr:Germany dbo:population ?population.
}

input= SPARQL Query for what is the population of France?
output= PREFIX dbo: <http://


In [16]:

prompt = """
input= SPARQL Query for what is the capital of Japan?
output=
"""

sequences = pipeline(
    prompt,
    max_length=300,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id = 50256
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: 
input= SPARQL Query for what is the capital of Japan?
output=
I have to write code in PHP?


#
# CodeGen-350M-multi


In [17]:
checkpoint = "Salesforce/codegen-350M-multi"


tokenizer = AutoTokenizer.from_pretrained(checkpoint)
pipeline = transformers.pipeline(
    "text-generation",
    model=checkpoint,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map=device,
)

In [18]:
prompt = "SPARQL Query for: what is the capital of Japan?"

sequences = pipeline(
    prompt,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: SPARQL Query for: what is the capital of Japan?
 *
 * @author  Benjamin Bengfort <benjamin@bengfort.com>
 * @author  Ricardo Cerqueira - 2015
 * @version $Id$
 *
 * @par
 * $Id$
 */
//------------------------------------------------------------------------------
// Copyright (C) 2013, The Regents of the University of California
// See README in root directory for license.
//------------------------------------------------------------------------------

#include "query.h"

//------------------------------------------------------------------------------
// Private methods
//------------------------------------------------------------------------------

/**
 * Return the capital of Japan.
 */
Query::Value Query::Capitals( const QString& query ) const
{
    // Build the Query object for the query
    Query q( query );

    // Create the JSON object
    q.SetJSON( true );

    // Retrieve the results
    QVariant


In [19]:

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query What is the capital of China?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Spain?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Spain dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Norway?
output=
"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query What is the capital of China?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Spain?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Spain dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Norway?
output=
 PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELEC

In [20]:

prompt = """
input= SPARQL Query for what is the capital of Thailand?
output=

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Thailand?
output=




In [ ]:
checkpoint = "EleutherAI/gpt-neo-2.7B"

#
# GPT-NEO-2.7B


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"


checkpoint = "EleutherAI/gpt-neo-2.7B"


tokenizer = AutoTokenizer.from_pretrained(checkpoint)
pipeline = transformers.pipeline(
    "text-generation",
    model=checkpoint,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map=device,
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [3]:
prompt = "SPARQL Query for: what is the capital of Japan?"

sequences = pipeline(
    prompt,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: SPARQL Query for: what is the capital of Japan?

I'm writing a program to search an RDF database and I need a way to get the capital of Japan. I don't know anything about RDF, but I know the following:

Japan is a country, which has a capital, which is Tokyo.
A country is made of a number of provinces, which is Japan's largest, so I would expect to find a query for "Japan", but not "Japan, the capital of".

So, I'm wondering if there is a way to search for "Japan, the capital of", in an RDF database (assuming the RDF database is in a JSON format) and if it exists, whether I should write my own query for it, or use a query that already exists?

A:

The query you are looking for is "Japan, the capital of". However, you should use the URI of Tokyo in your


In [4]:

prompt = """
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query What is the capital of China?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Spain?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Spain dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Norway?
output=
"""

sequences = pipeline(
    prompt,
    max_length=500,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Japan?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= SPARQL Query for What is the capital of Germany?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= SPARQL Query What is the capital of China?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Spain?
output= PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Spain dbo:capital ?capital .
}

input= SPARQL Query What is the capital of Norway?
output=
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT?capital 
WHERE {
  dbr:Norway dbo:capital?

In [5]:

prompt = """
input= SPARQL Query for what is the capital of Thailand?
output=

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Thailand?
output=

  <http://dbpedia.org/sparql>
  <http://dbpedia.org/sparql>

where <http://dbpedia.org/sparql> is the SPARQL endpoint of DBpedia, and


input: a SPARQL query
output:  the result of this query in the form of a graph


input= SPARQL Query for what is the capital of Thailand?
output=

  <http://dbpedia.org/sparql>
  <http://dbpedia.org/sparql>

where <http://dbpedia.org/sparql> is the SPARQL endpoint of DBpedia, and


output: the result of this query in the form of a graph


output=

  <http://dbpedia.org/sparql>

where <http://dbpedia.org/sparql> is the SPARQL endpoint of DBpedia

*/



In [8]:

prompt = """
input= SPARQL Query for what is the capital of Thailand?
output=

"""

sequences = pipeline(
    prompt,
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= SPARQL Query for what is the capital of Thailand?
output=



# The query below uses the same input and returns the same result as the
# query above.

SPARQL =





# The above query uses the "SPARQL" function to query SPARQL and returns
# the value "Thailand" in output.

spqr = spqr { 		
    "?sparql"   			
    "?sparql" 		
}




# The above query uses the "SPARQL" function to query SPARQL and returns
# null in output.

spqr null




    </textarea>
  </div>
  <script>
    var editor = CodeMirror.fromTextArea(document.getElementById("code2"), {
      mode: "text/x-sparql"
    });
  </script>

  <p><strong>MIME types defined:</strong> <code>application/sparql-query</code>.</p>

  </article>



In [11]:

prompt = """
input= Capital of Japan
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= Capital of United States
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:United_States dbo:capital ?capital .
}

input= Capital of China
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}

input= Capital of France
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:France dbo:capital ?capital .
}

# Capital of Germany
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= Capital of Canada
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Canada dbo:capital ?capital .
}

input= Capital of Australia
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Australia dbo:capital ?capital .
}

input= Capital of India
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:India dbo:capital ?capital .
}

input= Capital of Russia
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Russia dbo:capital ?capital .
}

input= Capital of Spain
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Spain dbo:capital ?capital .
}

input= Capital of Brazil
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Brazil dbo:capital ?capital .
}

# Capital of South Africa
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:South_Africa dbo:capital ?capital .
}

input= Capital of Italy
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Italy dbo:capital ?capital .
}

input= Capital of Mexico
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Mexico dbo:capital ?capital .
}

input= Capital of Argentina
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Argentina dbo:capital ?capital .
}

input= Capital of Turkey
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Turkey dbo:capital ?capital .
}

input= Capital of Iran
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Iran dbo:capital ?capital .
}

input= Capital of Egypt
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Egypt dbo:capital ?capital .
}

input= Capital of Indonesia
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Indonesia dbo:capital ?capital .
}

input= Capital of Sweden


"""

sequences = pipeline(
    prompt,
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 1254, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Result: 
input= Capital of Japan
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Japan dbo:capital ?capital .
}

input= Capital of United States
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:United_States dbo:capital ?capital .
}

input= Capital of China
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:China dbo:capital ?capital .
}

input= Capital of France
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:France dbo:capital ?capital .
}

# Capital of Germany
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?capital 
WHERE {
  dbr:Germany dbo:capital ?capital .
}

input= Capital of Canada
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/

In [13]:

prompt = """
input= Capital of Sweden

"""

sequences = pipeline(
    prompt,
#     max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

/home/mehrzad/anaconda3/envs/dbpedia/lib/python3.10/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
input= Capital of Sweden




In [15]:

prompt = """
Write an example fo sparql query over dbpedia
"""

sequences = pipeline(
    prompt,
#     max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
Write an example fo sparql query over dbpedia

I am new to sparqler. I am writing some example query in sparql to fetch the result of a query, and I am not able to get the exact output as given


In [16]:

prompt = """

Translate the following sentence into a SPARQL query: find capital of Japan

"""

sequences = pipeline(
    prompt,
#     max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 

Translate the following sentence into a SPARQL query: find capital of Japan

    SELECT?c?o WHERE{
   ?c <http://purl.org/dc/elements/1.


In [22]:

prompt = """

Translate the following sentence into a SPARQL query for DBpedia: find capital of Japan

"""

sequences = pipeline(
    prompt,
#     max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 

Translate the following sentence into a SPARQL query for DBpedia: find capital of Japan



?capital.name

?city.name

?population.n

?population.i

?population


In [20]:

prompt = """

Fill in the SPARQL query template to find the capital of a specific country. 
The country's name is 'Germany'.
The template is: 
'SELECT ?capital WHERE { dbr:" " dbo:capital ?capital . }'

"""

sequences = pipeline(
    prompt,
    max_length=300,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 

Fill in the SPARQL query template to find the capital of a specific country. 
The country's name is 'Germany'.
The template is: 
'SELECT ?capital WHERE { dbr:" " dbo:capital ?capital . }'

    SELECT * WHERE {

                         ?capital a geo:Point ;
                                                                 <-- 
    dbo:capital geo:Point.
                          }

This query will return 'Frankfurt'
I've been trying to use the SPARQL function ST_DWithin in order to find the capital of a specific country. 
In the following code, I've created a function to create an RDF query. 

#include <iostream>
#include <string>
#include <sparql/sparql_execution_context.h>
#include <spar


In [25]:

prompt = """

Translate the following question into SPARQL query towards DBpedia knowledge base using these prefixes:

PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>

Q: What is the capital of Japan
A:

"""

sequences = pipeline(
    prompt,
    max_length=00,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 

Translate the following question into SPARQL query towards DBpedia knowledge base using these prefixes:

PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>

Q: What is the capital of Japan
A:

                                                                                                                                                                                                                                    
